<a href="https://colab.research.google.com/github/aekanun2020/2025-AdvancedRAG/blob/main/comparingDiffChunking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# โค้ดตรวจจับรอยต่อระหว่าง Chunks (แก้ไขแล้ว)

# ติดตั้ง library ที่จำเป็น
!pip install langchain langchain-text-splitters

import re
import time
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
    CharacterTextSplitter,
    TokenTextSplitter,
    MarkdownHeaderTextSplitter
)

import re
import time
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
    CharacterTextSplitter,
    TokenTextSplitter,
    MarkdownHeaderTextSplitter
)

# สร้างไฟล์ตัวอย่าง
def create_sample_file():
    with open("sample_text.md", "w", encoding="utf-8") as f:
        f.write("""# รายงานการประชุมสามัญผู้ถือหุ้น ประจำปี 2567
ผ่านสื่ออิเล็กทรอนิกส์ (e-meeting)
ของ บริษัท เนชั่น กรุ๊ป (ไทยแลนด์) จำกัด (มหาชน)

## วัน เวลา และสถานที่ประชุม

การประชุมสามัญผู้ถือหุ้น ประจำปี 2567 ของบริษัท เนชั่น กรุ๊ป (ไทยแลนด์) จำกัด (มหาชน) ("บริษัทฯ")
ประชุมเมื่อวันศุกร์ที่ 19 เมษายน 2567 เวลา 14.00 น. ผ่านสื่ออิเล็กทรอนิกส์ (e-meeting) ณ ห้องประชุมสำนักงาน
ของบริษัท ชั้น 10 ถนนเทพรัตน แขวงบางนาใต้ เขตบางนา กรุงเทพมหานคร 10260

## กรรมการบริษัทที่เข้าประชุม และอยู่ในห้องประชุม มีจำนวน 2 ท่าน

1. นายมารุต ธรรคไกรวงศ์         ประธานกรรมการบริษัท
2. นายชนะชัย ลีนะบรรจง         กรรมการอิสระ กรรมการตรวจสอบ
และประธานกรรมการสรรหาและพิจารณาค่าตอบแทน

## กรรมการบริษัทที่เข้าประชุม ผ่านสื่ออิเล็กทรอนิกส์ มีจำนวน 4 ท่าน

1. นายศิริวุฒิ ทองคำ             กรรมการอิสระ และประธานกรรมการตรวจสอบ
2. นายชัยสิทธิ์ ภูวภิรมย์ขวัญ     กรรมการอิสระ กรรมการตรวจสอบ
ประธานกรรมการกำกับกิจการและความยั่งยืน
3. นายเจษฎา บูรณพันธุ์ศรี       กรรมการบริษัท และกรรมการกำกับกิจการและความยั่งยืน
4. นายกำพล แจ้งศิริ แลม         กรรมการบริษัท และกรรมการสรรหาและพิจารณาค่าตอบแทน""")

# อ่านไฟล์
def read_text_file():
    with open("sample_text.md", "r", encoding="utf-8") as f:
        text = f.read()
    return text

# ฟังก์ชันค้นหาข้อความที่ซ้ำกันระหว่างรอยต่อ (แก้ไขใหม่)
def find_overlapping_text(end_text, start_text, min_length=5):
    """
    ค้นหาข้อความที่ซ้ำซ้อนกันระหว่างส่วนท้ายของ chunk หนึ่งและส่วนต้นของ chunk ถัดไป

    Args:
        end_text: ส่วนท้ายของ chunk แรก
        start_text: ส่วนต้นของ chunk ถัดไป
        min_length: ความยาวขั้นต่ำของข้อความที่ซ้ำซ้อนกัน

    Returns:
        dict ที่มีข้อมูลเกี่ยวกับข้อความที่ซ้ำซ้อนกัน (หรือ None ถ้าไม่พบ)

    หมายเหตุ:
        การนับความยาวของข้อความภาษาไทยอาจไม่ตรงกับจำนวนตัวอักษรที่เห็น
        เนื่องจากสระบางตัวและวรรณยุกต์ถูกนับรวมกับตัวอักษร
    """
    # หาความยาวสูงสุดที่เป็นไปได้
    max_possible_length = min(len(end_text), len(start_text))

    # พิจารณาทุกความยาวที่เป็นไปได้ จากมากไปหาน้อย
    for length in range(max_possible_length, min_length - 1, -1):
        # ส่วนท้ายของ end_text ความยาว length
        end_suffix = end_text[-length:]
        # ส่วนต้นของ start_text ความยาว length
        start_prefix = start_text[:length]

        # ตรวจสอบว่าส่วนท้ายของ end_text ตรงกับส่วนต้นของ start_text หรือไม่
        if end_suffix == start_prefix and length >= min_length:
            return {
                'text': end_suffix,
                'length': length
            }

    # ถ้าไม่พบข้อความที่ซ้ำซ้อนกันที่มีความยาวอย่างน้อย min_length
    return None

# ทดสอบ splitter และแสดงผลรอยต่อระหว่าง chunks
def test_chunk_boundaries(chunk_size=1000, chunk_overlap=200):
    # สร้างไฟล์ตัวอย่าง
    create_sample_file()

    # อ่านข้อความ
    text = read_text_file()
    print(f"=================== ORIGINAL TEXT ===================")
    print(text)
    print(f"=====================================================")
    print(f"Text length: {len(text)} characters\n")

    # กำหนด splitters ที่ต้องการทดสอบ
    headers_to_split_on = [
        ("#", "header1"),
        ("##", "header2"),
        ("###", "header3"),
        ("####", "header4"),
    ]

    splitters = [
        (RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap),
         "LangChain RecursiveCharacterTextSplitter"),
        (CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap),
         "LangChain CharacterTextSplitter"),
        (TokenTextSplitter(chunk_size=chunk_size//4, chunk_overlap=chunk_overlap//4),
         "LangChain TokenTextSplitter"),
        (MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on),
         "LangChain MarkdownHeaderTextSplitter"),
    ]

    for splitter, name in splitters:
        print(f"\n====================================================================")
        print(f"=== Testing {name} ===")
        print(f"====================================================================")

        # อธิบายลักษณะและผลลัพธ์ที่คาดหวังของแต่ละวิธี
        if "RecursiveCharacterTextSplitter" in name:
            print(f"Parameters: chunk_size={chunk_size}, chunk_overlap={chunk_overlap}")
            print("Description: Splits text recursively by different separators (paragraphs, newlines, sentences, etc.)")
            print("Expected results: Should maintain the structure of paragraphs and sentences where possible.")
            print("                 Tends to split at natural boundaries like paragraph breaks or sentence ends.")
            print("                 We expect to see overlaps of approximately the chunk_overlap size.")

        elif "CharacterTextSplitter" in name:
            print(f"Parameters: chunk_size={chunk_size}, chunk_overlap={chunk_overlap}")
            print("Description: Splits text by character count without considering content structure.")
            print("Expected results: Will create uniform chunks of approximately chunk_size length.")
            print("                 May cut in the middle of sentences or even words.")
            print("                 Overlaps will be exactly the chunk_overlap size where possible.")

        elif "TokenTextSplitter" in name:
            print(f"Parameters: chunk_size={chunk_size//4}, chunk_overlap={chunk_overlap//4}")
            print("Description: Splits text based on token counts (similar to how LLMs process text).")
            print("Expected results: Will create chunks based on token count rather than character count.")
            print("                 Chunk size in characters may vary based on token encoding.")
            print("                 Good for maintaining token-level context for LLMs.")

        elif "MarkdownHeaderTextSplitter" in name:
            print(f"Parameters: headers_to_split_on={headers_to_split_on}")
            print("Description: Splits text based on Markdown headers (# or ##).")
            print("Expected results: Each chunk will start with a header and contain all content until the next header.")
            print("                 Chunks may vary greatly in size depending on document structure.")
            print("                 No overlaps between chunks, but maintains document hierarchy.")

        # แยกข้อความ
        if name == "LangChain MarkdownHeaderTextSplitter":
            docs = splitter.split_text(text)
            chunks = [doc.page_content for doc in docs]
        else:
            chunks = splitter.split_text(text)

        print(f"\nNumber of chunks: {len(chunks)}")

        # แสดงข้อมูลแต่ละ chunk
        for i, chunk in enumerate(chunks):
            print(f"\nChunk {i+1}/{len(chunks)} (Length: {len(chunk)}):")
            print(f"{chunk}")

        # วิเคราะห์รอยต่อระหว่าง chunks
        if len(chunks) > 1:
            print("\n--- Boundary Analysis ---")

            overlaps = []

            for i in range(len(chunks) - 1):
                chunk1 = chunks[i]
                chunk2 = chunks[i+1]

                # ใช้ส่วนท้ายและส่วนต้นที่ยาวพอสมควร
                end_section = chunk1[-min(300, len(chunk1)):]  # เพิ่มจำนวนตัวอักษรที่ตรวจสอบ
                start_section = chunk2[:min(300, len(chunk2))]  # เพิ่มจำนวนตัวอักษรที่ตรวจสอบ

                print(f"\nBoundary between chunks {i+1} and {i+2}:")
                print(f"End of chunk {i+1}: \"...{end_section[-100:]}\"")  # แสดงเฉพาะ 100 ตัวสุดท้าย
                print(f"Start of chunk {i+2}: \"{start_section[:100]}...\"")  # แสดงเฉพาะ 100 ตัวแรก

                # ค้นหาข้อความที่ซ้ำซ้อนกัน (ใช้ฟังก์ชันใหม่)
                overlap = find_overlapping_text(end_section, start_section)

                if overlap:
                    print("\nOverlap found:")
                    print(f"Text: \"{overlap['text']}\" (Length: {overlap['length']})")
                    overlaps.append(overlap['length'])
                else:
                    print("\nNo overlap found")
                    overlaps.append(0)

            # วิเคราะห์เพิ่มเติม
            print("\n--- Summary Analysis ---")

            avg_overlap = sum(overlaps) / len(overlaps) if overlaps else 0

            print(f"Average overlap length: {avg_overlap:.2f} characters")
            print(f"Min chunk size: {min(len(chunk) for chunk in chunks)} characters")
            print(f"Max chunk size: {max(len(chunk) for chunk in chunks)} characters")
            print(f"Avg chunk size: {sum(len(chunk) for chunk in chunks) / len(chunks):.2f} characters")

            if "RecursiveCharacterTextSplitter" in name:
                cuts_with_no_overlap = sum(1 for o in overlaps if o == 0)
                print(f"Cuts with no overlap: {cuts_with_no_overlap}/{len(overlaps)} ({cuts_with_no_overlap/len(overlaps)*100:.1f}%)")
                if avg_overlap < chunk_overlap * 0.5 and chunk_overlap > 0:
                    print("Note: Average overlap is significantly less than requested chunk_overlap")
                elif avg_overlap > 0:
                    print("Note: Overlap behavior generally matches expectations")

            elif "CharacterTextSplitter" in name:
                uniformity = max(len(chunk) for chunk in chunks) - min(len(chunk) for chunk in chunks)
                print(f"Chunk size uniformity (max-min): {uniformity} characters")
                if uniformity > chunk_size * 0.2:
                    print("Note: Chunks are less uniform than expected for CharacterTextSplitter")
                else:
                    print("Note: Chunk size uniformity matches expectations")

            elif "MarkdownHeaderTextSplitter" in name:
                # Check if chunks start with headers
                header_starts = sum(1 for chunk in chunks if chunk.lstrip().startswith('#'))
                print(f"Chunks starting with headers: {header_starts}/{len(chunks)} ({header_starts/len(chunks)*100:.1f}%)")
                if header_starts < len(chunks):
                    print("Note: Not all chunks start with headers as expected")
                else:
                    print("Note: Header-based splitting matches expectations")

# รันการทดสอบ
test_chunk_boundaries(chunk_size=500, chunk_overlap=100)

=================== ORIGINAL TEXT ===================
# รายงานการประชุมสามัญผู้ถือหุ้น ประจำปี 2567
ผ่านสื่ออิเล็กทรอนิกส์ (e-meeting)
ของ บริษัท เนชั่น กรุ๊ป (ไทยแลนด์) จำกัด (มหาชน)

## วัน เวลา และสถานที่ประชุม

การประชุมสามัญผู้ถือหุ้น ประจำปี 2567 ของบริษัท เนชั่น กรุ๊ป (ไทยแลนด์) จำกัด (มหาชน) ("บริษัทฯ")
ประชุมเมื่อวันศุกร์ที่ 19 เมษายน 2567 เวลา 14.00 น. ผ่านสื่ออิเล็กทรอนิกส์ (e-meeting) ณ ห้องประชุมสำนักงาน
ของบริษัท ชั้น 10 ถนนเทพรัตน แขวงบางนาใต้ เขตบางนา กรุงเทพมหานคร 10260

## กรรมการบริษัทที่เข้าประชุม และอยู่ในห้องประชุม มีจำนวน 2 ท่าน

1. นายมารุต ธรรคไกรวงศ์         ประธานกรรมการบริษัท
2. นายชนะชัย ลีนะบรรจง         กรรมการอิสระ กรรมการตรวจสอบ
และประธานกรรมการสรรหาและพิจารณาค่าตอบแทน

## กรรมการบริษัทที่เข้าประชุม ผ่านสื่ออิเล็กทรอนิกส์ มีจำนวน 4 ท่าน

1. นายศิริวุฒิ ทองคำ             กรรมการอิสระ และประธานกรรมการตรวจสอบ
2. นายชัยสิทธิ์ ภูวภิรมย์ขวัญ     กรรมการอิสระ กรรมการตรวจสอบ
ประธานกรรมการกำกับกิจการและความยั่งยืน
3. นายเจษฎา บูรณพันธุ์ศรี       กรรมการบริษัท แล